In [1]:
import xarray as xr
import zarr 
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import dask.array as da
import owslib.fes as fes
from owslib.etree import etree
from owslib.wfs import WebFeatureService
import geopandas as gpd
import shutil
import os
import zipfile
from osgeo import gdal, ogr, osr, gdal_array, gdalconst

### Dataset access and initial inspection

In [5]:
#replace with your online zarr storage for your personal zarr files
host= 'https://s3.waw3-1.cloudferro.com'
bucket = 'emodnet'

#BATHYMETRY
arco_location = 'bathymetry/bathymetry_2022.zarr'
dataset = xr.open_dataset(f"{host}/{bucket}/{arco_location}", engine='zarr')
dataset.attrs

# Inspect the Elevation data 
elevation_data_array = dataset.data_vars['elevation']
print(elevation_data_array)

<xarray.DataArray 'elevation' (latitude: 72000, longitude: 75840)> Size: 22GB
[5460480000 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float64 576kB 15.0 15.0 15.0 15.0 ... 90.0 90.0 90.0
  * longitude  (longitude) float64 607kB -36.0 -36.0 -36.0 ... 43.0 43.0 43.0
Attributes:
    grid_mapping:        crs
    long_name:           Elevation relative to sea level
    max:                 598.0
    min:                 -6912.60693359375
    sdn_parameter_name:  Sea-floor height (above Lowest Astronomical Tide dat...
    sdn_parameter_urn:   SDN:P01::HGHTALAT
    sdn_uom_name:        Metres
    sdn_uom_urn:         SDN:P06::ULAA
    standard_name:       geoid_height_above_reference_ellipsoid
    units:               m


### Open and Plot the Bathymetry raster data

In [9]:
# Define bounding boxes for your desired region within European waters 
region = 'North Sea' 
bbox = (-2, 8, 50, 60)

# Open the dataset
dataset = xr.open_dataset(f"{host}/{bucket}/{arco_location}", engine='zarr')

# Subset the dataset within the specified bounding box
subset_dataset = dataset.sel(latitude=slice(bbox[2], bbox[3]), longitude=slice(bbox[0], bbox[1]))

# Access the subset of the data variable containing elevation data
elevation_data_subset = subset_dataset['elevation']
print(elevation_data_subset) #Inspect the 'Elevation' data slice (subset)

# Create a map using Cartopy
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.coastlines()
ax.set_extent([bbox[0], bbox[1], bbox[2], bbox[3]])

# Plot elevation data, specifying the color bar range and colormap
elevation_plot = elevation_data_subset.plot(cmap = 'ocean', vmin=-500, vmax=50)  # Set color bar range from 0 to 500 meters
plt.title(f"Elevation in {region}")

#Save the figure locally 
#plt.savefig(f'{region}_DTM.tif')
plt.show()

<xarray.DataArray 'elevation' (latitude: 9600, longitude: 9600)> Size: 369MB
[92160000 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float64 77kB 50.0 50.0 50.0 50.0 ... 60.0 60.0 60.0
  * longitude  (longitude) float64 77kB -1.999 -1.998 -1.997 ... 7.998 7.999
Attributes:
    grid_mapping:        crs
    long_name:           Elevation relative to sea level
    max:                 598.0
    min:                 -6912.60693359375
    sdn_parameter_name:  Sea-floor height (above Lowest Astronomical Tide dat...
    sdn_parameter_urn:   SDN:P01::HGHTALAT
    sdn_uom_name:        Metres
    sdn_uom_urn:         SDN:P06::ULAA
    standard_name:       geoid_height_above_reference_ellipsoid
    units:               m


### Loading Vector data


In [ ]:
# Create Directory to download the Vector data
downloads_dir = 'shp_dir'
os.makedirs(downloads_dir, exist_ok=True) #Local on the service

#Connect to the EMODnet Human Activities WebFeatureService and Inspect
wfs = WebFeatureService('https://ows.emodnet-humanactivities.eu/wfs', version = '1.1.0')
print(wfs.identification.type)
print(wfs.identification.version)
print(wfs.identification.title)
print(wfs.identification.abstract)

WFS
1.1.0
GeoServer Web Feature Service
This is the reference implementation of WFS 1.0.0 and WFS 1.1.0, supports all WFS operations including Transaction.


In [ ]:
# OPTIONAL: Inspect available datasets
list(wfs.contents)

['emodnet:activelicenses',
 'emodnet:advisorycouncils',
 'emodnet:aquaculture',
 'emodnet:baltic',
 'emodnet:blacksea',
 'emodnet:longdistancefleet',
 'emodnet:market',
 'emodnet:mediterranean',
 'emodnet:northsea',
 'emodnet:northwesternwaters',
 'emodnet:outermostregions',
 'emodnet:pelagicstocks',
 'emodnet:southwesternwaters',
 'emodnet:aggregateareas',
 'emodnet:aggregates',
 'emodnet:fishingbeamtrawls',
 'emodnet:fishingbottomottertrawls',
 'emodnet:fishingbottomseines',
 'emodnet:fishingdredges',
 'emodnet:fishingpelagic',
 'emodnet:fishingstaticgears',
 'emodnet:fishingsubsurface',
 'emodnet:fishingsurface',
 'emodnet:barcelona',
 'emodnet:hydrocarbons',
 'emodnet:bucharest',
 'emodnet:desalination',
 'emodnet:dischargepoints',
 'emodnet:dredgespoil',
 'emodnet:dredgespoilpoly',
 'emodnet:dredging',
 'emodnet:munitions',
 'emodnet:munitionspoly',
 'emodnet:emeraldnetwork',
 'emodnet:faoareas',
 'emodnet:finfish',
 'emodnet:fishsales',
 'emodnet:divisioncatches',
 'emodnet:major

### Define desired vector dataset
Here we will access the shapefile containing polygons of all windfarms across European waters

In [ ]:
dataset = 'emodnet:windfarmspoly'

#Inspect Windfarm polygons dataset
print(wfs.contents[dataset].title)
print(wfs.contents[dataset].abstract)

Wind Farms (Polygons)
The dataset on offshore wind farms in the European seas was created in 2014 by CETMAR for the European Marine Observation and Data Network (EMODnet). It is the result of the aggregation and harmonization of datasets provided by several sources. It is updated every year and it is available for viewing and download on EMODnet web portal (Human Activities, https://emodnet.ec.europa.eu/en/human-activities). The dataset contains points and/or (where available) polygons representing offshore wind farms in the following countries: Belgium, Denmark, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Netherlands, Norway, Poland, Portugal, Spain, Sweden and United Kingdom. Each point and polygon has the following attributes (where available): Name, Nº of turbines, Status (Approved, Planned, Dismantled, Construction, Production, Test site), Country, Year, Power (MW), Distance to coast (metres) and Area (square kilometres). The distance to coast (EE

#### Subset the data with the North Sea Bounding box defined above


In [ ]:
response = wfs.getfeature(typename=dataset,
                          bbox=(bbox), #subset to the Study area ( based on bbox params)
                          outputFormat='application/json')

# Write it to a GeoJson file
with open(f'shp_dir/{dataset.replace(":","_")}.json', 'wb') as outfile:
    outfile.write(response.read())
response.close()

# Read the GeoJson file using GeoPandas
gdf = gpd.read_file(f'shp_dir/{dataset.replace(":","_")}.json')
gdf

,id,country,name,n_turbines,power_mw,status,year,dist_coast,area_sqkm,notes,geometry
0,windfarmspoly.54,Denmark,Nissum Bredning,4.0,28.0,Production,2018,552.097064,1.033175,NaN,"MULTIPOLYGON (((8.23407 56.67252, 8.25175 56.6..."
1,windfarmspoly.55,Denmark,Horns Rev II,91.0,209.3,Production,2009,28725.073617,31.377920,NaN,"MULTIPOLYGON (((7.59319 55.64797, 7.59999 55.6..."
2,windfarmspoly.49,Denmark,Samsa,10.0,23.0,Production,2003,3456.959776,0.882531,NaN,"MULTIPOLYGON (((10.58477 55.70969, 10.58460 55..."
3,windfarmspoly.50,Denmark,Horns Rev I,80.0,160.0,Production,2002,13863.488901,19.623752,NaN,"MULTIPOLYGON (((7.79637 55.50320, 7.80523 55.5..."
4,windfarmspoly.51,Denmark,Anholt,111.0,399.6,Production,2012,15488.518291,90.475307,NaN,"MULTIPOLYGON (((11.32309 56.57339, 11.31298 56..."
...,...,...,...,...,...,...,...,...,...,...,...
363,windfarmspoly.534,Germany,NaN,NaN,NaN,Planned,NaN,37564.345934,20.602709,NaN,"MULTIPOLYGON (((6.92272 54.09391, 6.92481 54.0..."
364,windfarmspoly.536,Germany,NaN,NaN,NaN,Planned,NaN,76210.545367,50.156457,NaN,"MULTIPOLYGON (((6.12636 54.29428, 6.12635 54.2..."
365,windfarmspoly.567,United Kingdom,IN Sinclair,NaN,NaN,Planned,NaN,61385.099323,25.260204,NaN,"MULTIPOLYGON (((-1.77773 58.28713, -1.75549 58..."
366,windfarmspoly.568,United Kingdom,IN Scaraben,NaN,NaN,Planned,NaN,58204.095337,33.082716,NaN,"MULTIPOLYGON (((-1.59915 58.24908, -1.59984 58..."


#### Store and unpack compressed (.zip) datasets

In [ ]:
# Define the output file paths
temp_zip_file = 'shp_dir/temp_shapefile.zip'
temp_dir = 'shp_dir/temp_shapefile'
output_shapefile ='shp_dir/temp_shapefile/windfarmspolyPolygon.shp'

# Create temporary directories
os.makedirs(temp_dir, exist_ok=True)

# Download and save the shapefile zip
response = wfs.getfeature(typename=dataset, outputFormat='SHAPE-ZIP')
with open(temp_zip_file, 'wb') as temp_file:
    shutil.copyfileobj(response, temp_file)

# Extract the zip file
with zipfile.ZipFile(temp_zip_file, 'r') as zip_ref:
    extracted_files = zip_ref.namelist()
    print(extracted_files)
    # Check if the output shapefile already exists before extraction
    if not os.listdir(temp_dir):
        zip_ref.extractall(temp_dir)

# Read the GeoDataFrame from the extracted Shapefile
Study_area = gpd.read_file(output_shapefile)
Study_area.to_file(output_shapefile, driver='ESRI Shapefile')

['windfarmspolyPolygon.shx', 'windfarmspolyPolygon.shp', 'windfarmspolyPolygon.cst', 'windfarmspolyPolygon.prj', 'windfarmspolyPolygon.dbf']


## Process the Shapefiles using GDAL
Here we reproject the shapefile to the correct coordinate reference system. This is relatively basic geoprocessing to inspect the capabilities of GDAL within the virtual machine.

In [ ]:
%%time
# Create the local folder for the reprojected shapefiles
reprojected = 'shp_dir/Repro_windfarms/'
os.makedirs(reprojected, exist_ok=True) #Local on the service

# Define a reprojecting function
def reproject_shapefile(input_shapefile, reprojected, target_epsg):
    """ Reprojects a shapefile to the specified EPSG code.
    Args:
        input_shapefile (str): Path to the input shapefile.
        output_shapefile (str): Path to save the reprojected shapefile.
        target_epsg (int): Target EPSG code for the projection.
    Returns:
        None
    """
    input_ds = ogr.Open(input_shapefile)
    input_layer = input_ds.GetLayer()
    target_spatial_ref = osr.SpatialReference()
    target_spatial_ref.ImportFromEPSG(target_epsg)
    transform = osr.CoordinateTransformation(input_layer.GetSpatialRef(), target_spatial_ref)
    
    # Create or open the output datasource
    if not os.path.exists(reprojected):
        os.makedirs(reprojected)
    output_ds = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource(reprojected)
    
    # Check if the layer already exists, and delete it if necessary
    layer_name = 'reprojected_shapefile'
    if output_ds.GetLayerByName(layer_name):
        output_ds.DeleteLayer(layer_name)
    
    # Create the output layer
    output_layer = output_ds.CreateLayer(layer_name, target_spatial_ref, geom_type=ogr.wkbPolygon)
    output_layer.CreateFields(input_layer.schema)
    
    for feature in input_layer:
        geometry = feature.GetGeometryRef()
        geometry.Transform(transform)
        new_feature = ogr.Feature(output_layer.GetLayerDefn())
        new_feature.SetGeometry(geometry)
        [new_feature.SetField(field.GetName(), feature.GetField(field.GetName())) for field in input_layer.schema]
        output_layer.CreateFeature(new_feature)
        new_feature = None
    
    input_ds = None
    output_ds = None

# Reproject the shapefile
reproject_shapefile(os.path.join(temp_dir,'windfarmspolyPolygon.shp'), reprojected, 4326)



c:\Users\matteo.mikos\AppData\Local\anaconda3\envs\ogc_workshop\Lib\site-packages\osgeo\ogr.py:560: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


CPU times: total: 125 ms
Wall time: 319 ms


## Plotting the Windfarm shapefile as an overlay of the Bathymetry raster

In [ ]:
%%time
from cartopy.feature import ShapelyFeature
from shapely.geometry import box
# Define bounding boxes for different regions within European waters
region = 'North Sea' 
bbox = (-2, 8, 50, 60)

# Plot on a map using Cartopy
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.coastlines()
ax.set_extent([bbox[0], bbox[1], bbox[2], bbox[3]])

# Read the reprojected shapefile into a GeoDataFrame
gdf_reprojected = gpd.read_file(reprojected)

# Plot elevation data with specific color bar range
elevation_plot = elevation_data_subset.plot(cmap='ocean', vmin=-500, vmax=0, ax=ax)  # Set color bar range from 0 to 500 meters
# Plot the GeoDataFrame
gdf_reprojected.plot(column='status',                                       # Specify the column in which the data you need to plot is located
         categorical=False,
         cmap='RdYlGn',
         legend=True,
         legend_kwds={'loc': 'lower right'},
         alpha=1,
         edgecolor = 'black', 
         linewidth = 0.3,
         ax=ax) 

plt.title(f"Bathymetry and Windfarm projects in the {region}")

#Save the figure locally 
plt.savefig(f'{region}.tif')
plt.show()


c:\Users\matteo.mikos\AppData\Local\anaconda3\envs\ogc_workshop\Lib\site-packages\geopandas\plotting.py:732: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(values.dtype):
